

1. Understanding LLM Evaluation:

    Explain why evaluating LLMs is more complex than traditional software.
    _____________
    Identify key reasons for evaluating an LLM’s safety.
    _____________
    Describe how adversarial testing contributes to LLM improvement.
    _____________
    Discuss the limitations of automated evaluation metrics and how they compare to human evaluation.


Sortie non déterministe : Contrairement aux logiciels traditionnels, les LLM peuvent produire différentes réponses pour la même entrée.

Multiplicité des réponses possibles : Il n’y a souvent pas une seule "bonne" réponse, mais plusieurs formulations valides.

______________________

Prévention des abus : Éviter que le modèle soit utilisé pour générer du contenu malveillant (désinformation, discours haineux, harcèlement, etc.).

Protection des utilisateurs : Éviter que le modèle donne des conseils dangereux ou des informations sensibles.

Alignement éthique : S’assurer que le modèle agit selon des principes éthiques et réglementaires (ex. RGPD, biais sociaux).

______________________

Détecter ses failles : Identifier les cas où il donne des réponses erronées, biaisées ou dangereuses.

Renforcer sa robustesse : En corrigeant les erreurs révélées par ces tests, on entraîne des modèles plus solides.

______________________

Limites des métriques automatiques :

    Manque de compréhension du contexte : Elles évaluent souvent mot à mot (BLEU, ROUGE) sans comprendre le sens.

    Insensibilité à la fluidité ou à la tonalité : Elles ne jugent pas si une réponse est naturelle ou appropriée.

    Incapacité à évaluer la factualité ou l’éthique : Elles ne détectent pas les erreurs factuelles ou les réponses biaisées.

    Réduction à des scores numériques : Elles donnent un chiffre qui ne reflète pas la richesse ou la qualité réelle d'une réponse.

Comparaison avec l’évaluation humaine :

    L’évaluation humaine est plus fiable, car elle tient compte du sens, du contexte, de l’utilité et de l’acceptabilité de la réponse.

    Mais elle est coûteuse et lente, ce qui la rend difficile à appliquer à grande échelle.

    La solution idéale est hybride : Utiliser des métriques automatiques pour des pré-filtrages rapides, puis compléter par des évaluations humaines ciblées.


Applying BLEU and ROUGE Metrics:

    Calculate the BLEU score for the following example:
        Reference: “Despite the increasing reliance on artificial intelligence in various industries, human oversight remains essential to ensure ethical and effective implementation.”
        Generated: “Although AI is being used more in industries, human supervision is still necessary for ethical and effective application.”

    Calculate the ROUGE score for the following example:
        Reference: “In the face of rapid climate change, global initiatives must focus on reducing carbon emissions and developing sustainable energy sources to mitigate environmental impact.”
        Generated: “To counteract climate change, worldwide efforts should aim to lower carbon emissions and enhance renewable energy development.”

    Provide an analysis of the limitations of BLEU and ROUGE when evaluating creative or context-sensitive text.

    Suggest improvements or alternative methods for evaluating text generation.

In [1]:
from math import sqrt, log, exp
from collections import Counter

In [23]:
def tokenize(text):
    # Simple tokenizer: lowercase and split on spaces
    return text.lower().replace(",", "").replace(".", "").split()

def get_ngrams(tokens, n):
    return Counter([' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)])

def modified_precision(hypothesis_tokens, reference_tokens, n):
    hyp_ngrams = get_ngrams(hypothesis_tokens, n)
    ref_ngrams = get_ngrams(reference_tokens, n)
    
    overlap = hyp_ngrams & ref_ngrams
    match_count = sum(overlap.values())
    total_count = max(sum(hyp_ngrams.values()), 1)
    
    if total_count == 0:
        return 0
    return match_count / total_count

def brevity_penalty(hypothesis_tokens, reference_tokens):
    c = len(hypothesis_tokens)
    r = len(reference_tokens)
    if c > r:
        return 1
    else:
        return exp(1 - r / c)

def compute_bleu(hypothesis, reference):
    hypothesis_tokens = tokenize(hypothesis)
    reference_tokens = tokenize(reference)
    
    precisions = []
    for n in range(1, 5):
        p_n = modified_precision(hypothesis_tokens, reference_tokens, n)
        # smoothing: if p_n == 0, use small epsilon
        if p_n == 0:
            p_n = 1e-8
        precisions.append(p_n)
    
    # Geometric mean of modified precisions
    log_precisions = [log(p) for p in precisions]
    geo_mean = exp(sum(log_precisions) / 4)
    
    # Brevity penalty
    bp = brevity_penalty(hypothesis_tokens, reference_tokens)
    
    bleu = bp * geo_mean
    return bleu, precisions, bp


In [24]:
# ==== Example ====
reference = "Despite the increasing reliance on artificial intelligence in various industries, human oversight remains essential to ensure ethical and effective implementation."
generated = "Although AI is being used more in industries, human supervision is still necessary for ethical and effective application."

bleu, p_ns, bp = compute_bleu(generated, reference)

print(f"BLEU score: {bleu:.3f}")
print(f"Precisions (p1 to p4): {[round(p, 3) for p in p_ns]}")
print(f"Brevity Penalty: {bp:.3f}")

BLEU score: 0.002
Precisions (p1 to p4): [0.333, 0.176, 0.062, 0.0]
Brevity Penalty: 0.895


In [14]:
#Autre tests

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Assure-toi que les ressources nécessaires sont téléchargées
nltk.download('punkt')

# Définition des phrases
reference = "Despite the increasing reliance on artificial intelligence in various industries, human oversight remains essential to ensure ethical and effective implementation."
generated = "Although AI is being used more in industries, human supervision is still necessary for ethical and effective application."

# Tokenisation simple
from nltk.tokenize import word_tokenize
ref_tokens = word_tokenize(reference)
gen_tokens = word_tokenize(generated)

# BLEU score avec lissage
chencherry = SmoothingFunction()
score = sentence_bleu([ref_tokens], gen_tokens, smoothing_function=chencherry.method1)

print(f"BLEU score: {score:.4f}")


c:\Users\alexa\AppData\Local\Programs\Python\Python312\Lib\site-packages\nltk\metrics\association.py:26: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  from scipy.stats import fisher_exact
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


BLEU score: 0.0779


In [21]:
%pip install rouge-score

from rouge_score import rouge_scorer

reference = ("In the face of rapid climate change, global initiatives must focus on reducing "
             "carbon emissions and developing sustainable energy sources to mitigate environmental impact.")

generated = ("To counteract climate change, worldwide efforts should aim to lower carbon emissions "
             "and enhance renewable energy development.")

# Initialize scorer for ROUGE-1, ROUGE-2, and ROUGE-L
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Score
scores = scorer.score(reference, generated)

# Print nicely
for key in scores:
    precision = scores[key].precision
    recall = scores[key].recall
    f1 = scores[key].fmeasure
    print(f"{key.upper()} -> Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}")


  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=25027 sha256=cf21a529d4d545b2072880ea616fdd1dbde8885e12f71dc3655ea9403fff9aa5
  Stored in directory: c:\users\alexa\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.
ROUGE1 -> Precision: 0.471, Recall: 0.333, F1: 0.390
ROUGE2 -> Precision: 0.188, Recall: 0.130, F1: 0.154
ROUGEL -> Precision: 0.353, Recall: 0.250, F1: 0.293


  DEPRECATION: Building 'rouge-score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge-score'. Discussion can be found at https://github.com/pypa/pip/issues/6334


Les métriques BLEU (traduction automatique) et ROUGE (résumé automatique) sont très utilisées dans l’évaluation de textes générés automatiquement. Cependant, elles ont de sérieuses limites lorsqu’on les applique à des textes créatifs ou sensibles au contexte, comme les dialogues, résumés complexes, ou textes littéraires.

C'est une comparaison purement lexical.
Aucune compréhension du sens sémantique et peu de correlation avec les jugement humains.
Dépendance à une ou plusieurs références.*
En Rouge, ordre des mot peu pris en compte.

 Alternatives et améliorations proposées

Métriques sémantiques à base d’embeddings
Évaluation avec un modèle de langage (LLM-as-a-judge)
Évaluation humaine (la référence absolue)


Model A: perplexity = 1/0.8 = 1.25
Model B: perplexity = 1/0.4 = 2.50

Model A > Model B en terme de perplixity (meilleur accurance)

Que signifie une perplexité égale à 100 ?

    Une perplexité de 100 indique que le modèle est aussi incertain que s’il devait choisir entre 100 mots possibles, tous équiprobables.

    Le modèle est donc assez peu performant, il prédit mal le mot suivant.

Pour améliorer, augmenter le nombre de données, faire du fine tunning, changer les hyperparamètres, de meilleurs données etc...

“Apologies, but comprehend I do not. Could you rephrase your question?”
C'est du vieux anglais donc (2/5) car gramaticalement correct.

MAIS EN LANGUAGE YODA: (4/5) car normalement c'est “Apologies, I offer. Comprehend, I do not. Rephrase your question, could you?”

⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣤⣤⠤⠐⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡌⡦⠊⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⣼⡊⢀⠔⠀⠀⣄⠤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣤⣤⣄⣀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⣶⠃⠉⠡⡠⠤⠊⠀⠠⣀⣀⡠⠔⠒⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣾⣿⢟⠿⠛⠛⠁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣼⡇⠀⠀⠀⠀⠑⠶⠖⠊⠁⠀⠀⠀⡀⠀⠀⠀⢀⣠⣤⣤⡀⠀⠀⠀⠀⠀⢀⣠⣤⣶⣿⣿⠟⡱⠁⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢰⣾⣿⡇⠀⢀⡠⠀⠀⠀⠈⠑⢦⣄⣀⣀⣽⣦⣤⣾⣿⠿⠿⠿⣿⡆⠀⠀⢀⠺⣿⣿⣿⣿⡿⠁⡰⠁⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣾⣿⣿⣧⣠⠊⣠⣶⣾⣿⣿⣶⣶⣿⣿⠿⠛⢿⣿⣫⢕⡠⢥⣈⠀⠙⠀⠰⣷⣿⣿⣿⡿⠋⢀⠜⠁⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⢿⣿⣿⣿⣿⣰⣿⣿⠿⣛⡛⢛⣿⣿⣟⢅⠀⠀⢿⣿⠕⢺⣿⡇⠩⠓⠂⢀⠛⠛⠋⢁⣠⠞⠁⠀⠀⠀⠀⠀⠀⠀⠀
⠘⢶⡶⢶⣶⣦⣤⣤⣤⣤⣤⣀⣀⣀⣀⡀⠀⠘⣿⣿⣿⠟⠁⡡⣒⣬⢭⢠⠝⢿⡡⠂⠀⠈⠻⣯⣖⣒⣺⡭⠂⢀⠈⣶⣶⣾⠟⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠙⠳⣌⡛⢿⣿⣿⣿⣿⣿⣿⣿⣿⣻⣵⣨⣿⣿⡏⢀⠪⠎⠙⠿⣋⠴⡃⢸⣷⣤⣶⡾⠋⠈⠻⣶⣶⣶⣷⣶⣷⣿⣟⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠈⠛⢦⣌⡙⠛⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡀⠀⠀⠩⠭⡭⠴⠊⢀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣿⣿⣿⣿⣿⡇⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠙⠓⠦⣄⡉⠛⠛⠻⢿⣿⣿⣿⣷⡀⠀⠀⠀⠀⢀⣰⠋⠀⠀⠀⠀⠀⣀⣰⠤⣳⣿⣿⣿⣿⣟⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠓⠒⠒⠶⢺⣿⣿⣿⣿⣦⣄⣀⣴⣿⣯⣤⣔⠒⠚⣒⣉⣉⣴⣾⣿⣿⣿⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠹⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣭⣉⣉⣤⣿⣿⣿⣿⣿⣿⡿⢀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⠟⡁⡆⠙⢶⣀⠀⢀⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣴⣶⣾⣿⣟⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠿⢛⣩⣴⣿⠇⡇⠸⡆⠙⢷⣄⠻⣿⣦⡄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣼⣿⣿⣿⣿⣿⣿⣿⣎⢻⣿⣿⣿⣿⣿⣿⣿⣭⣭⣭⣵⣶⣾⣿⣿⣿⠟⢰⢣⠀⠈⠀⠀⠙⢷⡎⠙⣿⣦⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣼⣿⣿⣿⣿⣿⣿⣿⣿⡟⣿⡆⢻⣿⣿⣿⣿⣿⣿⣿⣿⣿⠿⠿⠟⠛⠋⠁⢀⠇⢸⡇⠀⠀⠀⠀⠈⠁⠀⢸⣿⡆⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡜⡿⡘⣿⣿⣿⣿⣿⣶⣶⣤⣤⣤⣤⣤⣤⣤⣴⡎⠖⢹⡇⠀⠀⠀⠀⠀⠀⠀⠀⣿⣷⡄⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣦⡀⠘⢿⣿⣿⣿⣿⣿⣿⣿⣿⠿⠿⠛⠋⡟⠀⠀⣸⣷⣀⣤⣀⣀⣀⣤⣤⣾⣿⣿⣿⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⣸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣭⣓⡲⠬⢭⣙⡛⠿⣿⣿⣶⣦⣀⠀⡜⠀⠀⣰⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⢀⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣭⣛⣓⠶⠦⠥⣀⠙⠋⠉⠉⠻⣄⣀⣸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⣼⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣶⣆⠐⣦⣠⣷⠊⠁⠀⠀⡭⠙⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡆⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢠⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⢉⣛⡛⢻⡗⠂⠀⢀⣷⣄⠈⢆⠉⠙⠻⢿⣿⣿⣿⣿⣿⠇⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠘⣿⣿⡟⢻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡟⣉⢁⣴⣿⣿⣿⣾⡇⢀⣀⣼⡿⣿⣷⡌⢻⣦⡀⠀⠈⠙⠛⠿⠏⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⡄⠙⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠿⠛⠛⠛⢯⡉⠉⠉⠉⠉⠛⢼⣿⠿⠿⠦⡙⣿⡆⢹⣷⣤⡀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠿⠄⠈⠻⠿⠿⠿⠿⠿⠿⠛⠛⠿⠛⠉⠁⠀⠀⠀⠀⠀⠀⠻⠿⠿⠿⠿⠟⠉⠀⠀⠤⠴⠶⠌⠿⠘⠿⠿⠿⠿⠶⠤⠀⠀⠀⠀

Capital et non Capitol

Normalisation et correction orthographique automatique 

Compréhension contextuelle renforcée : Entraîner le modèle à repérer des erreurs fréquentes et à demander une clarification quand c'est ambigu

Demander une précision à l'utilisateur si nécessaire.


Pour les phrases qui testent la robustesse du LLM, il y'a : 

Who received the last Nobel Prize in Mathematics? -> Le Nobel n'existe pas

Which game received the Ace of Gold at Cannes in 2027? -> Données futurs

What is a "grolaxyis"? -> mot inventé


| Métrique               | Principe                                                                                                                            | Points forts                                                                               | Limites                                                                                                 |
| ---------------------- | ----------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------ | ------------------------------------------------------------------------------------------------------- |
| **ROUGE**              | Mesure le recouvrement des n-grammes (unigrammes, bigrammes, etc.) entre résumé généré et résumé de référence                       | Simple, rapide, standard pour résumé, corrèle assez bien avec qualité de résumé            | Sensible aux reformulations et paraphrases, pénalise la diversité lexicale                              |
| **BLEU**               | Mesure la précision des n-grammes du texte généré par rapport à une ou plusieurs références (principalement utilisée en traduction) | Utile pour la traduction, simple, standardisée                                             | Moins adaptée au résumé car favorise la correspondance exacte, pénalise reformulations et condensations |
| **BERTScore**          | Utilise des embeddings contextuels (BERT) pour mesurer la similarité sémantique entre résumé généré et référence                    | Capte la similarité sémantique même sans correspondance exacte, robuste aux reformulations | Coûteux en calcul, dépend du modèle utilisé, plus difficile à interpréter                               |
| **Évaluation humaine** | Jugement subjectif de la qualité par des annotateurs (fluency, fidélité, cohérence)                                                 | Plus fiable et complète, évalue la cohérence globale, la pertinence contextuelle           | Coûteuse, longue à mettre en place, variabilité inter-annotateurs                                       |
